# Testing out Google APis

I want to see how Google APIs work and how i can combine them together. 

Specifically I want to check out:

- Places API 
- StreetView API
 

I already got an API key and set it in my env file.

In [24]:
from dotenv import load_dotenv

load_dotenv()

True

Call the places API in a radius around Bologna:

In [25]:
import os
import requests
import json

api_key = os.getenv("GOOGLE_API_KEY")

# Example request to Places API - Text Search restricted to Bologna
query = "Giardini Margherita"
url = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    'X-Goog-FieldMask' : '*'   # many fields available, use '*' to get all
}

#useful: location contains lat and lon

# Bologna coordinates
bologna_lat = 44.4949
bologna_lng = 11.3426
radius_meters = 5000  # 5km radius

payload = {
    "textQuery": query,
    "locationBias": {
        "circle": {
            "center": {
                "latitude": bologna_lat,
                "longitude": bologna_lng
            },
            "radius": radius_meters
        }
    },
    "maxResultCount": 5  # Limit to 5 results
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

print("Status Code:", response.status_code)
print(f"\nSearching for '{query}' in Bologna (radius: {radius_meters}m)")
print(f"\nNumber of results: {len(data.get('places', []))}")
print("\nResponse:")
print(json.dumps(data, indent=2))


Status Code: 200

Searching for 'Giardini Margherita' in Bologna (radius: 5000m)

Number of results: 1

Response:
{
  "places": [
    {
      "name": "places/ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "id": "ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "types": [
        "hiking_area",
        "tourist_attraction",
        "sports_activity_location",
        "park",
        "point_of_interest",
        "establishment"
      ],
      "nationalPhoneNumber": "051 203040",
      "internationalPhoneNumber": "+39 051 203040",
      "formattedAddress": "Viale Giovanni Gozzadini, 40136 Bologna BO, Italy",
      "addressComponents": [
        {
          "longText": "Viale Giovanni Gozzadini",
          "shortText": "Viale Giovanni Gozzadini",
          "types": [
            "route"
          ],
          "languageCode": "it"
        },
        {
          "longText": "Bologna",
          "shortText": "Bologna",
          "types": [
            "locality",
            "political"
          ],
          "langu

Return only needed fields: for example location, which contains lat & lon

In [26]:
import os
import requests
import json

api_key = os.getenv("GOOGLE_API_KEY")

# Example request to Places API - Text Search restricted to Bologna
query = "Giardini Margherita"
url = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    'X-Goog-FieldMask' : 'places'   # many fields available, use '*' to get all
}

#useful: location contains lat and lon

# Bologna coordinates
bologna_lat = 44.4949
bologna_lng = 11.3426
radius_meters = 5000  # 5km radius

payload = {
    "textQuery": query,
    "locationBias": {
        "circle": {
            "center": {
                "latitude": bologna_lat,
                "longitude": bologna_lng
            },
            "radius": radius_meters
        }
    },
    "maxResultCount": 5  # Limit to 5 results
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

print("Status Code:", response.status_code)
print(f"\nSearching for '{query}' in Bologna (radius: {radius_meters}m)")
print(f"\nNumber of results: {len(data.get('places', []))}")
print("\nResponse:")
print(json.dumps(data, indent=2))
print("="*60)
print(data.get('places', [])[0].get('location', {}))


Status Code: 200

Searching for 'Giardini Margherita' in Bologna (radius: 5000m)

Number of results: 1

Response:
{
  "places": [
    {
      "name": "places/ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "id": "ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "types": [
        "hiking_area",
        "tourist_attraction",
        "sports_activity_location",
        "park",
        "point_of_interest",
        "establishment"
      ],
      "nationalPhoneNumber": "051 203040",
      "internationalPhoneNumber": "+39 051 203040",
      "formattedAddress": "Viale Giovanni Gozzadini, 40136 Bologna BO, Italy",
      "addressComponents": [
        {
          "longText": "Viale Giovanni Gozzadini",
          "shortText": "Viale Giovanni Gozzadini",
          "types": [
            "route"
          ],
          "languageCode": "it"
        },
        {
          "longText": "Bologna",
          "shortText": "Bologna",
          "types": [
            "locality",
            "political"
          ],
          "langu

In [27]:
import os
import requests

api_key = os.getenv("GOOGLE_API_KEY")

# Street View Static API URL
url = "https://maps.googleapis.com/maps/api/streetview"

# Use Piazza Maggiore coordinates (city center - has Street View coverage)
lat = 44.4949
lng = 11.3426

params = {
    "location": f"{lat},{lng}",  # Required: lat,lng coordinates
    "size": "640x640",            # Required: image size (max 640x640)
    "heading": 0,                 # Optional: camera direction in degrees (0-360)
    "pitch": 0,                   # Optional: vertical angle (-90 to 90, 0 = level)
    "fov": 90,                    # Optional: zoom level (10-120, higher = zoomed in)
    "key": api_key
}

response = requests.get(url, params=params)

print("Status Code:", response.status_code)
if response.status_code == 200:
    # Save the image
    with open("street_view_test.jpg", "wb") as f:
        f.write(response.content)
    print("Image saved as 'street_view_test.jpg'")
else:
    print(f"Error: {response.text}")

Status Code: 200
Image saved as 'street_view_test.jpg'


## Build the function 

We will write a function that samples `N` points around the radius `R` starting from a fixed location `center`.

It will need to retry because we may encounter locations that do not have street views associated; we will implement this later on.

In [28]:
import random
import math

def sample_points_in_radius(center : tuple[float, float], radius : float, N : int) -> list[tuple[float, float]]:
    """
    Sample N points in a circle of radius R centered at center.

    Args:
        center: tuple of float, the center of the circle (lat, lon)
        radius: float, the radius of the circle in meters
        N: int, the number of points to sample

    Returns:
        list of tuples of float, the sampled points (lat, lon)
    """
    center_lat, center_lon = center
    points = []
    
    for _ in range(N):
        # Generate random angle and distance
        angle = random.uniform(0, 2 * math.pi)
        # Use square root for uniform distribution in circle
        distance = math.sqrt(random.uniform(0, 1)) * radius
        
        # Convert meters to approximate lat/lon offset
        # Rough approximation: 1 degree lat ≈ 111,000m, 1 degree lon ≈ 111,000m * cos(lat)
        lat_offset = distance / 111000
        lon_offset = distance / (111000 * math.cos(math.radians(center_lat)))
        
        # Calculate new coordinates
        new_lat = center_lat + lat_offset * math.cos(angle)
        new_lon = center_lon + lon_offset * math.sin(angle)
        
        points.append((new_lat, new_lon))
    
    return points
    

In [29]:
# Test the function
bologna_center = (44.4949, 11.3426)  # Piazza Maggiore
radius_meters = 1000  # 1km radius
n_points = 5

sampled_points = sample_points_in_radius(bologna_center, radius_meters, n_points)

print(f"Center: {bologna_center}")
print(f"Radius: {radius_meters}m")
print(f"Sampled {len(sampled_points)} points:")
for i, (lat, lon) in enumerate(sampled_points):
    print(f"  {i+1}: ({lat:.6f}, {lon:.6f})")


Center: (44.4949, 11.3426)
Radius: 1000m
Sampled 5 points:
  1: (44.490024, 11.349147)
  2: (44.488250, 11.340896)
  3: (44.499599, 11.334680)
  4: (44.499558, 11.349062)
  5: (44.492670, 11.346296)


In [30]:
# Visualize with folium
import folium

# Create map centered on Bologna
m = folium.Map(location=bologna_center, zoom_start=13)

# Add the center point with a different marker
folium.Marker(
    bologna_center,
    popup='Center (Piazza Maggiore)',
    tooltip='Center',
    icon=folium.Icon(color='red', icon='flag')
).add_to(m)

# Add the sampled points
for i, (lat, lon) in enumerate(sampled_points):
    folium.Marker(
        [lat, lon],
        popup=f'Point {i+1}',
        tooltip=f'Point {i+1}',
        icon=folium.Icon(color='blue', icon='circle')
    ).add_to(m)

# Add a circle to show the radius
folium.Circle(
    location=bologna_center,
    radius=radius_meters,
    color='red',
    fill=False,
    weight=2,
    opacity=0.5
).add_to(m)

# Display the map
m


In [ ]:
# Build Street View URL entries and save to CSV (URLs without API key)
import os
import pandas as pd

# Configuration
size = "640x640"
fov = 90
headings = [0, 90, 180, 270]
pitches = [-45, 0, 45]
include_key_in_csv = False  # avoid exposing API key in the CSV
api_key = os.getenv("GOOGLE_API_KEY")

streetview_base = "https://maps.googleapis.com/maps/api/streetview"

rows = []
for point_idx, (lat, lon) in enumerate(sampled_points):
    for heading in headings:
        for pitch in pitches:
            params = {
                "size": size,
                "location": f"{lat},{lon}",
                "heading": heading,
                "pitch": pitch,
                "fov": fov,
            }
            # URL without the key (safe to store/share)
            url_no_key = (
                f"{streetview_base}?size={params['size']}&location={params['location']}"
                f"&heading={params['heading']}&pitch={params['pitch']}&fov={params['fov']}"
            )
            # Template with placeholder for key (for agents that can inject it)
            url_with_placeholder = url_no_key + "&key={API_KEY}"
            # Optionally materialize with your key (not recommended to persist)
            url_with_key = (
                url_no_key + f"&key={api_key}" if (include_key_in_csv and api_key) else None
            )

            rows.append({
                "point_index": point_idx,
                "lat": lat,
                "lon": lon,
                "heading": heading,
                "pitch": pitch,
                "fov": fov,
                "size": size,
                "url_template": url_with_placeholder,
                "url_no_key": url_no_key,
                "url_with_key": url_with_key,
            })

sv_df = pd.DataFrame(rows)
print(f"Generated {len(sv_df)} Street View entries for {len(sampled_points)} points.")
sv_df.head()


In [ ]:
# Save to CSV
out_csv = "streetview_samples.csv"
sv_df.to_csv(out_csv, index=False)
print(f"Saved {len(sv_df)} rows to {out_csv}")
